# Benchmark for bayesian optimization

In [28]:
from test_functions import Hart6
from bayes_lib import ExactGPModel, train_hyper_params
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.constraints import Interval
import torch
import matplotlib.pyplot as plt

In [95]:
from scipy.stats import truncnorm
def get_test_x(n, mu=0.5, sigma=0.25, lower=0.0, upper=1.0):
    """ returns a n x d tensor containing random points taking from truncated 
    normal distributions of means mu, and standard deviation sigma (mu and sigma are floats or lists of length d).
    The distribution is truncated between lower and upper bounds.
    for more information on the truncated normal distribution see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
Parameters
==============
n: int
    number of points to return
mu: float or array-like, default: 0.5
    list of means of length d
sigma: float or array-like, default: 0.25
    list of standard deviations (must be the same length as mu)
lower: float, default: 0.0
    lower bound
uper: float, default: 1.0

Returns
==============
tensor of shape (n, d)

    """
    if type(mu) == int or type(mu) == float:
        mu = [mu]
    if type(sigma) == int or type(sigma) == float:
        sigma = [sigma]
    if len(mu) != len(sigma):
        raise ValueError(f"mu and sigma must have the same length, but got len(mu)={len(mu)}, len(sigma)={len(sigma)}")
        
    ndim = len(mu)
    out = []
    for i in range(ndim):
        X = truncnorm(
            (lower - mu[i]) / sigma[i], (upper - mu[i]) / sigma[i], loc=mu[i], scale=sigma[i])
        out.append(torch.from_numpy(X.rvs(n)))

    return torch.stack(out).T

In [96]:
# Bayes opt loop
min_iter = 2 
max_iter = 5
it = 0
Model = ExactGPModel
error_func = Hart6()
likelihood = GaussianLikelihood(noise_constraint=Interval(0.0,1e-14))

train_x = torch.rand(12).reshape(-1,6)
train_y = error_func.f(train_x)
n = 21 # number of evaluation points of the test function in one direction
# test_x = torch.stack([torch.linspace(0, 1, 21) for i in range(6)]).T
test_x = get_test_x(3, mu=[0.5 for i in range(6)],  sigma=[0.25 for i in range(6)])
test_x.shape

torch.Size([3, 6])

torch.Size([1000000, 6])

In [24]:
while (it<min_iter or (train_x[-1] - train_x[-2]).abs()>1e-6) and it<max_iter:
    # Run the forward model with hyperparam opt
    model = Model(train_x, train_y, likelihood)
    train_hyper_params(model, likelihood)

    # Run the aquisition method
    EI = ExpectedImprovement(model, best_f=0.2)
    ei = EI(test_x.reshape(-1,1,1))

#     # Add the suggested point to the training points
#     x_new = test_x[ei.argmax().item()]
#     train_x = torch.cat((train_x.reshape(-1,1), x_new.reshape(-1,1))).reshape(-1)
#     train_y = torch.cat((train_y.reshape(-1,1), f(x_new).reshape(-1,1))).reshape(-1)
#     print(f"{it}/{max_iter}: ...")
#     it += 1

AttributeError: 'numpy.ndarray' object has no attribute 'to'